In [1]:
import d_expanse.d_exp_simple as dxp
from gen_utils import k_clustering as k_means
import seaborn as sn
import pandas as pd

D-EXPANSE module initialised


In [3]:
model, country_df, tech_df = dxp.run_d_expanse()

KeyError: 0

In [ ]:
elec_demand = (
        country_df.loc["ElSupplied_annual_central"]["Value"]
        - tech_df.loc["Actual_generation", "Storage"]
        - country_df.loc["Distribution_losses"]["Value"]
    )
k_ratio_y_d, demand_y_d_h = k_means.get_demand_shape(model.Years, model.DxpDays, elec_demand)
elec_supplied_ratio_y0 = k_means.get_supplied_ratio_y_d(demand_y_d_h, k_ratio_y_d, model.DxpDays, model.Years.first())

In [ ]:
imp_cap_y0 = model.imp_p_nom["Import",1990].value
exp_cap_y0 = model.exp_p_nom["Export",1990].value
line_cap_y0 = model.line_p_nom["Line",1990].value
print("Import cap:", imp_cap_y0,
      "Export cap:", exp_cap_y0,
      "Line cap:", line_cap_y0, "in GW")

In [ ]:
hourly = pd.DataFrame()
hourly["Import"] = pd.Series(model.imp_p.extract_values(), name=model.imp_p.name)
hourly["Export"] = model.exp_p.extract_values().values()
hourly.plot()


In [ ]:
yearly = pd.DataFrame(columns=hourly.columns)
# hourly.droplevel(0).index.get_level_values(0)
names = hourly.index.get_level_values(0).unique()
years = model.Years
days = model.DxpDays
hours = model.DxpHours
for c in hourly.columns:
    for n in names:
        for y in years:
            yearly.loc[y,c] = sum([hourly.loc[("Import", y,d,h),c]/ 1000*(365 * k_ratio_y_d[y][d]) for d in model.DxpDays for h in model.DxpHours])

In [ ]:
yearly.plot()

In [ ]:
trade_cap = pd.DataFrame()
trade_cap["DXP Import"] = yearly["Import"]
trade_cap["DXP Export"] = yearly["Export"]
trade_cap["Actual Import"] = country_df.loc["Actual_import"]["Value"]
trade_cap["Actual Export"] = country_df.loc["Actual_export"]["Value"]
trade_cap.plot()

In [ ]:
trade_cap["Actual Imp+Exp"] = trade_cap["Actual Import"] + trade_cap["Actual Export"]
trade_cap["DXP Imp+Exp"] = trade_cap["DXP Import"] + trade_cap["DXP Export"]
ax = trade_cap.plot.line()
ax.legend(bbox_to_anchor=(1.1, 1.05))

In [ ]:
trade_cap["Max 'actual' trade capacity"] = tech_df.loc["Actual_capacity"]["Import"]*365*24/1000
trade_cap["Actual Imp+Exp"] = trade_cap["Actual Import"] + trade_cap["Actual Export"]
trade_cap["DXP Imp+Exp"] = trade_cap["DXP Import"] + trade_cap["DXP Export"]
trade_cap["2010 ENTSO-e Pmax"] = 9.9*365*24/1000 # 9 GW
trade_cap["2012 ENTSO-e NTC MAX"] = 6.54*365*24/1000 # 
ax = trade_cap.plot.line()
ax.legend(bbox_to_anchor=(1.1, 1.05))

In [ ]:
model.line_p_nom.pprint()

In [1]:
import data.unit_conversions as conversion
import pandas as pd

country='AUT'
technologies=['HardCoal']

years=range(1990,2020)

#Read technology files
input_files=[]
for t in technologies:
    input_files.append(pd.read_csv('/Users/ruiziv/switchdrive/ACCURACY/D-EXPANSE/AUT/Technologies/{0}_{1}.csv'.format(country,t),skiprows=4))
input_df=pd.concat(input_files)

In [2]:
###Convert units in technology dataframe and reindex to match the current df/Excel format
converted_df=input_df.apply(conversion.convert_units,new_power='GW', new_energy='TWh', axis=1)
converted_df.to_csv("/Users/ruiziv/Downloads/just_energy.csv")


In [3]:
final_df=converted_df.apply(conversion.convert_currency,new_cy='EUR', deflator_country='local', axis=1)
final_df.to_csv("/Users/ruiziv/Downloads/cost_energy.csv")